In [1]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [16]:
open_park = spark.read.csv("C:\\Users\\ariha\\Desktop\\Ai\\switch2.0\\open_violation.csv",header=True).limit(100)

In [17]:
parking_voilation = spark.read.csv("C:\\Users\\ariha\\Desktop\\Ai\\switch2.0\\Parking_Violations_Issued_-_Fiscal_Year_2016.csv",header=True).limit(100)

In [27]:
parking_voilation.printSchema()
open_park.printSchema()

root
 |-- Summons Number: string (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Code: string (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: string (nullable = true)
 |-- Street Code2: string (nullable = true)
 |-- Street Code3: string (nullable = true)
 |-- Vehicle Expiration Date: string (nullable = true)
 |-- Violation Location: string (nullable = true)
 |-- Violation Precinct: string (nullable = true)
 |-- Issuer Precinct: string (nullable = true)
 |-- Issuer Code: string (nullable = true)
 |-- Issuer Command: string (nullable = true)
 |-- Issuer Squad: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Time First Observed: string (nullable = true)
 |-- Violation County: str

## Task 1
write a Spark program that finds all parking violations that have been paid, i.e., that do not occur in openviolations.csv.

In [26]:
# filter_the amount_due =0
open_parking_amount_due_zero= open_park.filter(open_park["Amount Due"]=="$0.00")

In [34]:
fines_paid= parking_voilation.join(open_parking_amount_due_zero,parking_voilation["Summons Number"]== \
                                   open_parking_amount_due_zero["Summons Number"],"inner")
# fines_paid.printSchema()
fines_paid.select(fines_paid["plate id"],fines_paid["violation precinct"],fines_paid["violation code"])


DataFrame[plate id: string, violation precinct: string, violation code: string]

##  Task2

Write a Spark program that finds the distribution of the violation types, i.e., for each violation code, the 
number of violations that have this code.

In [42]:
alpha = parking_voilation.groupBy("Violation Code").count()
alpha.show()

+--------------+-----+
|Violation Code|count|
+--------------+-----+
|            46|    2|
|            21|   63|
|            75|    1|
|            71|    9|
|            41|    2|
|            70|    3|
|            20|    4|
|            38|    2|
|            67|    1|
|            37|    6|
|            40|    2|
|            74|    2|
|            18|    1|
|            68|    2|
+--------------+-----+



## Task 3:
Write a Spark program that finds the total and average amount due in open violations for each license 


In [43]:
open_park.printSchema()

root
 |-- Plate: string (nullable = true)
 |-- State: string (nullable = true)
 |-- License Type: string (nullable = true)
 |-- Summons Number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Violation: string (nullable = true)
 |-- Judgment Entry Date: string (nullable = true)
 |-- Fine Amount: string (nullable = true)
 |-- Penalty Amount: string (nullable = true)
 |-- Interest Amount: string (nullable = true)
 |-- Reduction Amount: string (nullable = true)
 |-- Payment Amount: string (nullable = true)
 |-- Amount Due: string (nullable = true)
 |-- Precinct: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Violation Status: string (nullable = true)
 |-- Summons Image: string (nullable = true)



In [74]:
new_pen_violation=open_park.withColumn("Amount_Due",open_park["Amount Due"].cast("int"))
new_pen_violation.printSchema()

root
 |-- Plate: string (nullable = true)
 |-- State: string (nullable = true)
 |-- License Type: string (nullable = true)
 |-- Summons Number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Violation: string (nullable = true)
 |-- Judgment Entry Date: string (nullable = true)
 |-- Fine Amount: string (nullable = true)
 |-- Penalty Amount: string (nullable = true)
 |-- Interest Amount: string (nullable = true)
 |-- Reduction Amount: string (nullable = true)
 |-- Payment Amount: string (nullable = true)
 |-- Amount Due: string (nullable = true)
 |-- Precinct: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Violation Status: string (nullable = true)
 |-- Summons Image: string (nullable = true)
 |-- Amount_Due: integer (nullable = true)



In [95]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum,avg,max
from pyspark.sql import functions as F
# new_pen_violation.groupBy("License Type").agg(F.mean('Amount_Due'), F.sum('Amount_Due')).show()
a= new_pen_violation.groupBy("License Type").agg(avg('Amount_Due'), sum('Amount_Due')).show()

                          



+------------+---------------+---------------+
|License Type|avg(Amount_Due)|sum(Amount_Due)|
+------------+---------------+---------------+
|         PAS|           null|           null|
|         999|           null|           null|
|         COM|           null|           null|
|         OMT|           null|           null|
|         OMS|           null|           null|
+------------+---------------+---------------+



# Task 4

This was correct

Write a Spark program that computes the total number of violations for vehicles registered in the state of 
NY and all other vehicles

In [83]:
open_park.printSchema()

root
 |-- Plate: string (nullable = true)
 |-- State: string (nullable = true)
 |-- License Type: string (nullable = true)
 |-- Summons Number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Violation: string (nullable = true)
 |-- Judgment Entry Date: string (nullable = true)
 |-- Fine Amount: string (nullable = true)
 |-- Penalty Amount: string (nullable = true)
 |-- Interest Amount: string (nullable = true)
 |-- Reduction Amount: string (nullable = true)
 |-- Payment Amount: string (nullable = true)
 |-- Amount Due: string (nullable = true)
 |-- Precinct: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Violation Status: string (nullable = true)
 |-- Summons Image: string (nullable = true)



# tASK 5

In [96]:
open_park_1=open_park.groupBy("Plate","State").count()

In [102]:
open_park_1.printSchema()

root
 |-- Plate: string (nullable = true)
 |-- State: string (nullable = true)
 |-- count: long (nullable = false)



In [105]:
a=open_park_1.groupby().max('count')

In [108]:
a.show()

+----------+
|max(count)|
+----------+
|         7|
+----------+



## Task 6